### Installing libraries

In [ ]:
library(ggplot2)
library(jsonlite)
library(data.table)
library(asreml)
library(dplyr)

### Reading in files, converting variable types

In [ ]:
# Dataset for ASREML-R models
phenotype <- read.csv("updated_data_2013_2014.csv", header=T)
phenotype$dam <- as.factor(phenotype$dam)
phenotype$sire <- as.factor(phenotype$sire)
phenotype$year <- as.factor(phenotype$year)
phenotype$hybrid <- as.factor(phenotype$hybrid)
# phenotype$egg_mass <- as.numeric(levels(phenotype$egg_mass))[phenotype$egg_mass]
# phenotype$egg_diameter <- as.numeric(levels(phenotype$egg_diameter))[phenotype$egg_diameter]
phenotype$temp <- as.factor(phenotype$temp)

# Converting empty cells to NA, omitting NAs from dataset
phenotype[phenotype==""] <- NA
phenotype <- na.omit(phenotype)

# Dataset for reaction norm figures
grouped_data <- phenotype %>% group_by(sire, temp) %>% summarize(pct.male = mean(sex == "1"))
grouped_data$temp <- as.factor(grouped_data$temp)
grouped_data$sire <- as.factor(grouped_data$sire)
grouped_data$temp = as.numeric(as.character(grouped_data$temp)) 

### Trait analyses

#### 1. Egg mass

In [ ]:
# Running ASREML-R models
model.1 <- asreml(fixed = egg_mass ~ hybrid, random = ~year + sire + sire/dam, data=phenotype)
model.2 <- asreml(fixed = egg_mass ~ hybrid, random = ~year + sire, data=phenotype)
model.3 <- asreml(fixed = egg_mass ~ hybrid, random = ~year + sire/dam, data=phenotype)
model.4 <- asreml(fixed = egg_mass ~ hybrid, random = ~sire + sire/dam, data=phenotype)

# Likelihood ratio tests for testing random effect significance
paste0(2 * (model.1$loglik - model.2$loglik))
paste0(2 * (model.1$loglik - model.3$loglik))
paste0(2 * (model.1$loglik - model.4$loglik))

# Wald test for fixed effect significance
wald(model.1, denDF="numeric", ssType="conditional")

# Calculate narrow-sense heritability (h2) from basic.1 model
# h2 = (4 * sire variance) / (total variance)
vpredict(egg_mass_model, heritability ~4*V2 / (V1 + V2 + V3 + V4))

# calculate maximum maternal effects (m2) from basic.1 model
# assuming dominance variation (d2) = 0, m2max = dam[sire] - sire 
vpredict(model.1, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

#### 2. Egg diamter

In [ ]:
# Running ASREML-R models
model.1 <- asreml(fixed = egg_diameter ~ hybrid, random = ~year + sire + sire/dam, data=phenotype)
model.2 <- asreml(fixed = egg_diameter ~ hybrid, random = ~year + sire, data=phenotype)
model.3 <- asreml(fixed = egg_diameter ~ hybrid, random = ~year + sire/dam, data=phenotype)
model.4 <- asreml(fixed = egg_diameter ~ hybrid, random = ~sire + sire/dam, data=phenotype)

# Likelihood ratio tests for testing random effect significance
paste0(2 * (model.1$loglik - model.2$loglik))
paste0(2 * (model.1$loglik - model.3$loglik))
paste0(2 * (model.1$loglik - model.4$loglik))

# Wald test for fixed effect significance
wald(model.1, denDF="numeric", ssType="conditional")

# Calculate narrow-sense heritability (h2) from basic.1 model
# h2 = (4 * sire variance) / (total variance)
vpredict(model.1, heritability ~4*V2 / (V1 + V2 + V3 + V4))

# calculate maximum maternal effects (m2) from basic.1 model
# assuming dominance variation (d2) = 0, m2max = dam[sire] - sire 
vpredict(model.1, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

#### 3. Hatchling mass

In [ ]:
# Running ASREML-R models
# The tempeature main effect as well as genotype x temperature interactions are present in models

# Running ASREML-R models
model.1 <- asreml(fixed = hatch_mass ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
model.2 <- asreml(fixed = hatch_mass ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp, data=phenotype)
model.3 <- asreml(fixed = hatch_mass ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire/dam:temp, data=phenotype)
model.4 <- asreml(fixed = hatch_mass ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire:temp + sire/dam:temp, data=phenotype)
model.5 <- asreml(fixed = hatch_mass ~ hybrid + hybrid:temp + temp, random = ~year + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
model.6 <- asreml(fixed = hatch_mass ~ hybrid + hybrid:temp + temp, random = ~sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)

# Likelihood ratio tests for testing random effect significance
paste0(2 * (model.1$loglik - model.2$loglik))
paste0(2 * (model.1$loglik - model.3$loglik))
paste0(2 * (model.1$loglik - model.4$loglik))
paste0(2 * (model.1$loglik - model.5$loglik))
paste0(2 * (model.1$loglik - model.6$loglik))

# Wald test for fixed effect significance
wald(model.1, denDF="numeric", ssType="conditional")

# Calculating h2 and m2 at individual temperatures
# 22 degree models, year is not a variable

# Removing 22.5, 26, 27, and 28 degree temperatures from dataset
phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

model.22.A <- asreml(fixed = hatch_mass ~ hybrid, random = ~sire + sire/dam, data=phenotypeA) 
model.22.B <- asreml(fixed = hatch_mass ~ hybrid, random = ~sire, data=phenotypeA) 
model.22.C <- asreml(fixed = hatch_mass ~ hybrid, random = ~sire/dam, data=phenotypeA) 

# paste0(2 * (model.22.A$loglik - model.22.B$loglik))
# paste0(2 * (model.22.A$loglik - model.22.C$loglik))

# vpredict(model.22.A, heritability ~4*V1 / (V1 + V2 + V3))
# vpredict(model.22.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# 22.5 degree models, year is not an effect

# Removing 22, 26, 27, and 28 degree temperatures 
# phenotypeE <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.5.A <- asreml(fixed = hatch_mass ~ hybrid, random = ~sire + sire/dam, data=phenotypeE)
# model.22.5.B <- asreml(fixed = hatch_mass ~ hybrid, random = ~sire, data=phenotypeE)
# model.22.5.C <- asreml(fixed = hatch_mass ~ hybrid, random = ~sire/dam, data=phenotypeE)

# paste0(2 * (model.22.5.A$loglik - model.22.5.B$loglik))
# paste0(2 * (model.22.5.A$loglik - model.22.5.C$loglik))

# vpredict(model.22.5.A, heritability ~4*V1 / (V1 + V2 + V3))
# vpredict(model.22.5.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# 26 degree models
# Removing 22, 22.5, 27 and 28 degree temperatures 
# phenotypeB <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==27 | phenotype$temp==28),]

# model.26.A <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeB)
# model.26.B <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire, data=phenotypeB)
# model.26.C <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire/dam, data=phenotypeB)

# paste0(2 * (model.26.A$loglik - model.26.B$loglik))
# paste0(2 * (model.26.A$loglik - model.26.C$loglik))

# # summary(model.26.A)$varcomp


# # vpredict(model.26.A, heritability ~4*V2 / (V1 + V2 + V3 + V4)) 
# # vpredict(model.26.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))



# # 27 degree models
# # Removing 22, 22.5, 26 and 28 degree temperatures 
# phenotypeC <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==28),]

# model.27.A <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeC)
# model.27.B <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire, data=phenotypeC)
# model.27.C <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire/dam, data=phenotypeC)

# paste0(2 * (model.27.A$loglik - model.27.B$loglik))
# paste0(2 * (model.27.A$loglik - model.27.C$loglik))

# # summary(model.27.A)$varcomp

# # vpredict(model.27.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # vpredict(model.27.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))


# # 28 degree models
# # # Removing 22, 22.5, 26 and 27 degree temperatures 
# phenotypeD <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27),]

# model.28.A <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeD)
# model.28.B <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire, data=phenotypeD)
# model.28.C <- asreml(fixed = hatch_mass ~ hybrid, random = ~year + sire/dam, data=phenotypeD)

# paste0(2 * (model.28.A$loglik - model.28.B$loglik))
# paste0(2 * (model.28.A$loglik - model.28.C$loglik))

# # summary(model.28.A)$varcomp


# # vpredict(model.28.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # vpredict(model.28.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))


#### 4. Hatchling carapace length

In [ ]:
# Running ASREML-R models
model.1 <- asreml(fixed = hatch_cl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
model.2 <- asreml(fixed = hatch_cl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp, data=phenotype)
# # model.3 <- asreml(fixed = hatch_cl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire/dam:temp, data=phenotype)
# # model.4 <- asreml(fixed = hatch_cl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire:temp + sire/dam:temp, data=phenotype)
# # model.5 <- asreml(fixed = hatch_cl ~ hybrid + hybrid:temp + temp, random = ~year + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
# # model.6 <- asreml(fixed = hatch_cl ~ hybrid + hybrid:temp + temp, random = ~sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)

# # paste0(2 * (model.1$loglik - model.2$loglik))
# # paste0(2 * (model.1$loglik - model.3$loglik))
# # paste0(2 * (model.1$loglik - model.4$loglik))
# # paste0(2 * (model.1$loglik - model.5$loglik))
# # paste0(2 * (model.1$loglik - model.6$loglik))

# # wald(hatch_CL_model, denDF="numeric", ssType="conditional")




# # # 22 degree models, no year effect
# # # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.A <- asreml(fixed = hatch_cl ~ hybrid, random = ~sire + sire/dam, data=phenotypeA) 
# # model.22.B <- asreml(fixed = hatch_cl ~ hybrid, random = ~sire, data=phenotypeA) 
# # model.22.C <- asreml(fixed = hatch_cl ~ hybrid, random = ~sire/dam, data=phenotypeA)

# # paste0(2 * (model.22.A$loglik - model.22.B$loglik))
# # paste0(2 * (model.22.A$loglik - model.22.C$loglik)) 

# summary(model.22.A)$varcomp

# # vpredict(model.22.A, heritability ~4*V1 / (V1 + V2 + V3))
# # vpredict(model.22.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))


# # 22.5 degree models, no year effect

# # Removing 22, 26, 27, and 28 degree temperatures 

# phenotypeE <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.5.A <- asreml(fixed = hatch_cl ~ hybrid, random = ~sire + sire/dam, data=phenotypeE)
# # model.22.5.B <- asreml(fixed = hatch_cl ~ hybrid, random = ~sire, data=phenotypeE)
# # model.22.5.C <- asreml(fixed = hatch_cl ~ hybrid, random = ~sire/dam, data=phenotypeE)

# # paste0(2 * (model.22.5.A$loglik - model.22.5.B$loglik))
# # paste0(2 * (model.22.5.A$loglik - model.22.5.C$loglik))

# # # # vpredict(model.22.5.A, heritability ~4*V1 / (V1 + V2 + V3))
# # # # vpredict(model.22.5.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# summary(model.22.5.A)$varcomp


# # 26 degree models
# # Removing 22, 22.5, 27 and 28 degree temperatures 
# phenotypeB <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==27 | phenotype$temp==28),]

# model.26.A <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeB)
# # model.26.B <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire, data=phenotypeB)
# # model.26.C <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire/dam, data=phenotypeB)

# # paste0(2 * (model.26.A$loglik - model.26.B$loglik))
# # paste0(2 * (model.26.A$loglik - model.26.C$loglik))

# # # # vpredict(model.26.A, heritability ~4*V2 / (V1 + V2 + V3 + V4)) 
# # # # vpredict(model.26.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.26.A)$varcomp


# # 27 degree models
# # Removing 22, 22.5, 26 and 28 degree temperatures 
# phenotypeC <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==28),]

# model.27.A <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeC)
# # model.27.B <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire, data=phenotypeC)
# # model.27.C <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire/dam, data=phenotypeC)

# # paste0(2 * (model.27.A$loglik - model.27.B$loglik))
# # paste0(2 * (model.27.A$loglik - model.27.C$loglik))

# # # # vpredict(model.27.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # # # vpredict(model.27.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.27.A)$varcomp


# # 28 degree models
# # # Removing 22, 22.5, 26 and 27 degree temperatures 
# phenotypeD <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27),]

# model.28.A <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeD)
# model.28.B <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire, data=phenotypeD)
# model.28.C <- asreml(fixed = hatch_cl ~ hybrid, random = ~year + sire/dam, data=phenotypeD)

# paste0(2 * (model.28.A$loglik - model.28.B$loglik))
# paste0(2 * (model.28.A$loglik - model.28.C$loglik))

# # # vpredict(model.28.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # # vpredict(model.28.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# # summary(model.28.A)$varcomp



#### 5. Hatchling plastron length

In [ ]:
# ### 4. hatch PL ###
# # summary(hatch_PL_model)$varcomp
# # wald(hatch_PL_model, denDF="numeric", ssType="conditional")

# model.1 <- asreml(fixed = hatch_pl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
# model.2 <- asreml(fixed = hatch_pl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp, data=phenotype)
# model.3 <- asreml(fixed = hatch_pl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire/dam:temp, data=phenotype)
# model.4 <- asreml(fixed = hatch_pl ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire:temp + sire/dam:temp, data=phenotype)
# model.5 <- asreml(fixed = hatch_pl ~ hybrid + hybrid:temp + temp, random = ~year + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
# model.6 <- asreml(fixed = hatch_pl ~ hybrid + hybrid:temp + temp, random = ~sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)

# summary(model.1)$loglik 
# summary(model.2)$loglik
# summary(model.3)$loglik
# summary(model.4)$loglik
# summary(model.5)$loglik
# summary(model.6)$loglik



# summary(model.1)
# # paste0(2 * (model.1$loglik - model.2$loglik))
# # paste0(2 * (model.1$loglik - model.3$loglik))
# # paste0(2 * (model.1$loglik - model.4$loglik))
# # paste0(2 * (model.1$loglik - model.5$loglik))
# # paste0(2 * (model.1$loglik - model.6$loglik))


# # # 22 degree models, no year effect
# # # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.A <- asreml(fixed = hatch_pl ~ hybrid, random = ~sire + sire/dam, data=phenotypeA) 
# model.22.B <- asreml(fixed = hatch_pl ~ hybrid, random = ~sire, data=phenotypeA) 
# model.22.C <- asreml(fixed = hatch_pl ~ hybrid, random = ~sire/dam, data=phenotypeA) 

# paste0(2 * (model.22.A$loglik - model.22.B$loglik))
# paste0(2 * (model.22.A$loglik - model.22.C$loglik))

# # summary(model.22.A)$varcomp

# # vpredict(model.22.A, heritability ~4*V1 / (V1 + V2 + V3))
# # vpredict(model.22.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))


# # 22.5 degree models, no year effect

# # Removing 22, 26, 27, and 28 degree temperatures 

# phenotypeE <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.5.A <- asreml(fixed = hatch_pl ~ hybrid, random = ~sire + sire/dam, data=phenotypeE)
# model.22.5.B <- asreml(fixed = hatch_pl ~ hybrid, random = ~sire, data=phenotypeE)
# model.22.5.C <- asreml(fixed = hatch_pl ~ hybrid, random = ~sire/dam, data=phenotypeE)

# paste0(2 * (model.22.5.A$loglik - model.22.5.B$loglik))
# paste0(2 * (model.22.5.A$loglik - model.22.5.C$loglik))

# # # vpredict(model.22.5.A, heritability ~4*V1 / (V1 + V2 + V3))
# # # vpredict(model.22.5.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# # summary(model.22.5.A)$varcomp



# # 26 degree models
# # Removing 22, 22.5, 27 and 28 degree temperatures 
# phenotypeB <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==27 | phenotype$temp==28),]

# model.26.A <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeB)
# model.26.B <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire, data=phenotypeB)
# model.26.C <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire/dam, data=phenotypeB)

# paste0(2 * (model.26.A$loglik - model.26.B$loglik))
# paste0(2 * (model.26.A$loglik - model.26.C$loglik))

# # # vpredict(model.26.A, heritability ~4*V2 / (V1 + V2 + V3 + V4)) 
# # # vpredict(model.26.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# # summary(model.26.A)$varcomp


# # 27 degree models
# # Removing 22, 22.5, 26 and 28 degree temperatures 
# phenotypeC <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==28),]

# model.27.A <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeC)
# model.27.B <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire, data=phenotypeC)
# model.27.C <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire/dam, data=phenotypeC)

# paste0(2 * (model.27.A$loglik - model.27.B$loglik))
# paste0(2 * (model.27.A$loglik - model.27.C$loglik))

# # # vpredict(model.27.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # # vpredict(model.27.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# # summary(model.27.A)$varcomp


# # 28 degree models
# # # Removing 22, 22.5, 26 and 27 degree temperatures 
# phenotypeD <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27),]

# model.28.A <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeD)
# model.28.B <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire, data=phenotypeD)
# model.28.C <- asreml(fixed = hatch_pl ~ hybrid, random = ~year + sire/dam, data=phenotypeD)

# paste0(2 * (model.28.A$loglik - model.28.B$loglik))
# paste0(2 * (model.28.A$loglik - model.28.C$loglik))

# # # vpredict(model.28.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # # vpredict(model.28.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# # summary(model.28.A)$varcomp

#### 6. Incubation temperature

In [ ]:
# ### 6. Incubation duration ###

# # summary(inc_dur_model)$varcomp
# # wald(inc_dur_model, denDF="numeric", ssType="conditional")

# model.1 <- asreml(fixed = inc_dur ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
# model.2 <- asreml(fixed = inc_dur ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp, data=phenotype)
# model.3 <- asreml(fixed = inc_dur ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire/dam:temp, data=phenotype)
# model.4 <- asreml(fixed = inc_dur ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire:temp + sire/dam:temp, data=phenotype)
# model.5 <- asreml(fixed = inc_dur ~ hybrid + hybrid:temp + temp, random = ~year + sire/dam + sire:temp + sire/dam:temp, data=phenotype)
# model.6 <- asreml(fixed = inc_dur ~ hybrid + hybrid:temp + temp, random = ~sire + sire/dam + sire:temp + sire/dam:temp, data=phenotype)

# paste0(2 * (model.1$loglik - model.2$loglik))
# paste0(2 * (model.1$loglik - model.3$loglik))
# paste0(2 * (model.1$loglik - model.4$loglik))
# paste0(2 * (model.1$loglik - model.5$loglik))
# paste0(2 * (model.1$loglik - model.6$loglik))

# # # 22 degree models, no year effect
# # # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.A <- asreml(fixed = inc_dur ~ hybrid, random = ~sire + sire/dam, data=phenotypeA) 
# model.22.B <- asreml(fixed = inc_dur ~ hybrid, random = ~sire, data=phenotypeA) 
# model.22.C <- asreml(fixed = inc_dur ~ hybrid, random = ~sire/dam, data=phenotypeA) 

# paste0(2 * (model.22.A$loglik - model.22.B$loglik))
# paste0(2 * (model.22.A$loglik - model.22.C$loglik))

# # summary(model.22.A)$varcomp

# # vpredict(model.22.A, heritability ~4*V1 / (V1 + V2 + V3))
# # vpredict(model.22.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))


# # 22.5 degree models, no year effect

# # Removing 22, 26, 27, and 28 degree temperatures 

# phenotypeE <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.5.A <- asreml(fixed = inc_dur ~ hybrid, random = ~sire + sire/dam, data=phenotypeE)
# model.22.5.B <- asreml(fixed = inc_dur ~ hybrid, random = ~sire, data=phenotypeE)
# model.22.5.C <- asreml(fixed = inc_dur ~ hybrid, random = ~sire/dam, data=phenotypeE)

# paste0(2 * (model.22.5.A$loglik - model.22.5.B$loglik))
# paste0(2 * (model.22.5.A$loglik - model.22.5.C$loglik))

# # summary(model.22.5.A)$varcomp


# # vpredict(model.22.5.A, heritability ~4*V1 / (V1 + V2 + V3))
# # vpredict(model.22.5.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))



# # 26 degree models
# # Removing 22, 22.5, 27 and 28 degree temperatures 
# phenotypeB <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==27 | phenotype$temp==28),]

# model.26.A <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeB)
# model.26.B <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire, data=phenotypeB)
# model.26.C <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire/dam, data=phenotypeB)

# paste0(2 * (model.26.A$loglik - model.26.B$loglik))
# paste0(2 * (model.26.A$loglik - model.26.C$loglik))

# # # vpredict(model.26.A, heritability ~4*V2 / (V1 + V2 + V3 + V4)) 
# # # vpredict(model.26.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# # summary(model.26.A)$varcomp


# # 27 degree models
# # Removing 22, 22.5, 26 and 28 degree temperatures 
# phenotypeC <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==28),]

# model.27.A <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeC)
# model.27.B <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire, data=phenotypeC)
# model.27.C <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire/dam, data=phenotypeC)

# paste0(2 * (model.27.A$loglik - model.27.B$loglik))
# paste0(2 * (model.27.A$loglik - model.27.C$loglik))

# # # vpredict(model.27.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # # vpredict(model.27.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# # summary(model.27.A)$varcomp


# # 28 degree models
# # # Removing 22, 22.5, 26 and 27 degree temperatures 
# phenotypeD <- phenotype[!(phenotype$temp==22 | phenotype$temp=="shift" | phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27),]

# model.28.A <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeD)
# model.28.B <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire, data=phenotypeD)
# model.28.C <- asreml(fixed = inc_dur ~ hybrid, random = ~year + sire/dam, data=phenotypeD)

# paste0(2 * (model.28.A$loglik - model.28.B$loglik))
# paste0(2 * (model.28.A$loglik - model.28.C$loglik))

# # vpredict(model.28.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# # vpredict(model.28.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.28.A)$varcomp


#### 7. Sex as a continuous trait

In [ ]:
### 7. sex ###

# summary(sex_model)$varcomp
# wald(sex_model, denDF="numeric", ssType="conditional")

# model.1 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp + (sire/dam):temp, data=phenotype)
# model.2 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp, data=phenotype)
# model.3 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + (sire/dam):temp, data=phenotype)
# model.4 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire:temp + (sire/dam):temp, data=phenotype)
# model.5 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire/dam + sire:temp + (sire/dam):temp, data=phenotype)
# model.6 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~sire + sire/dam + sire:temp + (sire/dam):temp, data=phenotype)

# # # summary(model.1)

# paste0(2 * (model.1$loglik - model.2$loglik))
# paste0(2 * (model.1$loglik - model.3$loglik))
# paste0(2 * (model.1$loglik - model.4$loglik))
# paste0(2 * (model.1$loglik - model.5$loglik))
# paste0(2 * (model.1$loglik - model.6$loglik))

# summary(model.1)$varcomp
# summary(model.2)$varcomp
# summary(model.3)$varcomp
# summary(model.4)$varcomp
# summary(model.5)$varcomp
# summary(model.6)$varcomp



# # 22 degree models, no year effect
# # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]


# model.22.A <- asreml(fixed = sex ~ hybrid, random = ~sire + sire/dam, data=phenotypeA) 
# # # # # # # # # model.22.B <- asreml(fixed = sex ~ hybrid, random = ~sire, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # # # model.22.C <- asreml(fixed = sex ~ hybrid, random = ~sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 

# summary(model.22.A)$varcomp

# vpredict(model.22.A, heritability ~4*V1 / (V1 + V2 + V3))
# vpredict(model.22.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# LRT1 = 2 * (model.22.A$loglik - model.22.B$loglik)
# LRT2 = 2 * (model.22.A$loglik - model.22.C$loglik)

# paste0("LRT(model.22.A-model.22.B): ", LRT1)
# paste0("LRT(model.22.A-model.22.C): ", LRT2)

# # 22.5 degree models, no year effect
# # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.5.A <- asreml(fixed = sex ~ hybrid, random = ~sire + sire/dam, data=phenotypeA) 
# # # # # # # model.22.5.B <- asreml(fixed = sex ~ hybrid, random = ~sire, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # model.22.5.C <- asreml(fixed = sex ~ hybrid, random = ~sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# # summary(model.22.5.A)$varcomp

# vpredict(model.22.5.A, heritability ~4*V1 / (V1 + V2 + V3))
# vpredict(model.22.5.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# LRT1 = 2 * (model.22.5.A$loglik - model.22.5.B$loglik)
# LRT2 = 2 * (model.22.5.A$loglik - model.22.5.C$loglik)

# paste0(LRT1)
# paste0(LRT2)


# # temp shift models 
# # Removing 22, 22.5, 26, 27, and 28 degree temperatures 

phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

model.shift.A <- asreml(fixed = sex ~ 1, random = ~year + sire + dam, data=phenotype) 
# # # model.shift.B <- asreml(fixed = sex ~ 1, random = ~year + sire, data=phenotype, family = asr_binomial(link=logit))
# # # model.shift.C <- asreml(fixed = sex ~ 1, random = ~year + dam, data=phenotype, family = asr_binomial(link=logit)) 

# summary(model.shift.A)$varcomp

vpredict(model.shift.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
vpredict(model.shift.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# ywt.vp <- cbind.data.frame(names(model.shift.A$vparameters),vpc.char(model.shift.A),
# 1:length(model.shift.A$vparameters.type))
# names(ywt.vp) <- c("names", "constraint", "number");ywt.vp 

# summary(model.shift.A)$varcomp
# summary(model.shift.B)$varcomp
# summary(model.shift.C)$varcomp


# LRT1 = 2 * (model.shift.A$loglik - model.shift.B$loglik)
# LRT2 = 2 * (model.shift.A$loglik - model.shift.C$loglik)

# paste0(LRT1)
# paste0(LRT2)


# # 26 degree models 
# # Removing 22, 22.5, shift, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==27 | phenotype$temp==28),]

# model.26.A <- asreml(fixed = sex ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeA) 
# # # # # # # # model.26.B <- asreml(fixed = sex_ratio ~ hybrid, random = ~year + sire, data=phenotypeA, family = asr_binomial(link=logit))
# # # # # # # # model.26.C <- asreml(fixed = sex_ratio ~ hybrid, random = ~year + sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# # ywt.vp <- cbind.data.frame(names(model.26.A$vparameters),vpc.char(model.26.A),
# # 1:length(model.26.A$vparameters.type))
# # names(ywt.vp) <- c("names", "constraint", "number");ywt.vp 


# vpredict(model.26.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# vpredict(model.26.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.26.A)$varcomp

# LRT1 = 2 * (model.26.A$loglik - model.26.B$loglik)
# LRT2 = 2 * (model.26.A$loglik - model.26.C$loglik)

# paste0(LRT1)
# paste0(LRT2)


# # 27 degree models 
# # Removing 22, 22.5, 26, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==28),]

# model.27.A <- asreml(fixed = sex ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeA) 
# # # # # # # model.27.B <- asreml(fixed = sex ~ hybrid, random = ~year + sire, data=phenotypeA, family = asr_binomial(link=logit))
# # # # # # # model.27.C <- asreml(fixed = sex ~ hybrid, random = ~year + sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# vpredict(model.27.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# vpredict(model.27.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))


# summary(model.27.A)$varcomp

# LRT1 = 2 * (model.27.A$loglik - model.27.B$loglik)
# LRT2 = 2 * (model.27.A$loglik - model.27.C$loglik)

# paste0(LRT1)
# paste0(LRT2)



# # 28 degree models 
# # Removing 22, 22.5, 26, and 27 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==27),]

# model.28.A <- asreml(fixed = sex ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeA) 
# # # # # # # model.28.B <- asreml(fixed = sex ~ hybrid, random = ~year + sire, data=phenotypeA, family = asr_binomial(link=logit))
# # # # # # # model.28.C <- asreml(fixed = sex ~ hybrid, random = ~year + sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# vpredict(model.28.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# vpredict(model.28.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.28.A)$varcomp

# LRT1 = 2 * (model.28.A$loglik - model.28.B$loglik)
# LRT2 = 2 * (model.28.A$loglik - model.28.C$loglik)

# paste0(LRT1)
# paste0(LRT2)

#### 7. Sex as a binary trait

In [ ]:
### 7. sex ###

# summary(sex_model)$varcomp
# wald(sex_model, denDF="numeric", ssType="conditional")

# model.1 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp + (sire/dam):temp, data=phenotype)
# model.2 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + sire:temp, data=phenotype)
# model.3 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire/dam + (sire/dam):temp, data=phenotype)
# model.4 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire + sire:temp + (sire/dam):temp, data=phenotype)
# model.5 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~year + sire/dam + sire:temp + (sire/dam):temp, data=phenotype)
# model.6 <- asreml(fixed = sex ~ hybrid + hybrid:temp + temp, random = ~sire + sire/dam + sire:temp + (sire/dam):temp, data=phenotype)

# # # summary(model.1)

# paste0(2 * (model.1$loglik - model.2$loglik))
# paste0(2 * (model.1$loglik - model.3$loglik))
# paste0(2 * (model.1$loglik - model.4$loglik))
# paste0(2 * (model.1$loglik - model.5$loglik))
# paste0(2 * (model.1$loglik - model.6$loglik))

# summary(model.1)$varcomp
# summary(model.2)$varcomp
# summary(model.3)$varcomp
# summary(model.4)$varcomp
# summary(model.5)$varcomp
# summary(model.6)$varcomp



# # 22 degree models, no year effect
# # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]


# model.22.A <- asreml(fixed = sex ~ hybrid, random = ~sire + sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # # # model.22.B <- asreml(fixed = sex ~ hybrid, random = ~sire, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # # # model.22.C <- asreml(fixed = sex ~ hybrid, random = ~sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 

# summary(model.22.A)$varcomp

# vpredict(model.22.A, heritability ~4*V1 / (V1 + V2 + V3))
# vpredict(model.22.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# LRT1 = 2 * (model.22.A$loglik - model.22.B$loglik)
# LRT2 = 2 * (model.22.A$loglik - model.22.C$loglik)

# paste0("LRT(model.22.A-model.22.B): ", LRT1)
# paste0("LRT(model.22.A-model.22.C): ", LRT2)

# # 22.5 degree models, no year effect
# # Removing 22.5, 26, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

# model.22.5.A <- asreml(fixed = sex ~ hybrid, random = ~sire + sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # model.22.5.B <- asreml(fixed = sex ~ hybrid, random = ~sire, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # model.22.5.C <- asreml(fixed = sex ~ hybrid, random = ~sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# # summary(model.22.5.A)$varcomp

# vpredict(model.22.5.A, heritability ~4*V1 / (V1 + V2 + V3))
# vpredict(model.22.5.A, maternal_effects ~(V2 - V1) / (V1 + V2 + V3))

# LRT1 = 2 * (model.22.5.A$loglik - model.22.5.B$loglik)
# LRT2 = 2 * (model.22.5.A$loglik - model.22.5.C$loglik)

# paste0(LRT1)
# paste0(LRT2)


# # temp shift models 
# # Removing 22, 22.5, 26, 27, and 28 degree temperatures 

phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==27 | phenotype$temp==28),]

model.shift.A <- asreml(fixed = sex ~ 1, random = ~year + sire + dam, data=phenotype, family = asr_binomial(link=logit)) 
# # # model.shift.B <- asreml(fixed = sex ~ 1, random = ~year + sire, data=phenotype, family = asr_binomial(link=logit))
# # # model.shift.C <- asreml(fixed = sex ~ 1, random = ~year + dam, data=phenotype, family = asr_binomial(link=logit)) 

# summary(model.shift.A)$varcomp

vpredict(model.shift.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
vpredict(model.shift.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# ywt.vp <- cbind.data.frame(names(model.shift.A$vparameters),vpc.char(model.shift.A),
# 1:length(model.shift.A$vparameters.type))
# names(ywt.vp) <- c("names", "constraint", "number");ywt.vp 

# summary(model.shift.A)$varcomp
# summary(model.shift.B)$varcomp
# summary(model.shift.C)$varcomp


# LRT1 = 2 * (model.shift.A$loglik - model.shift.B$loglik)
# LRT2 = 2 * (model.shift.A$loglik - model.shift.C$loglik)

# paste0(LRT1)
# paste0(LRT2)


# # 26 degree models 
# # Removing 22, 22.5, shift, 27, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==27 | phenotype$temp==28),]

# model.26.A <- asreml(fixed = sex ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # # model.26.B <- asreml(fixed = sex_ratio ~ hybrid, random = ~year + sire, data=phenotypeA, family = asr_binomial(link=logit))
# # # # # # # # model.26.C <- asreml(fixed = sex_ratio ~ hybrid, random = ~year + sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# # ywt.vp <- cbind.data.frame(names(model.26.A$vparameters),vpc.char(model.26.A),
# # 1:length(model.26.A$vparameters.type))
# # names(ywt.vp) <- c("names", "constraint", "number");ywt.vp 


# vpredict(model.26.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# vpredict(model.26.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.26.A)$varcomp

# LRT1 = 2 * (model.26.A$loglik - model.26.B$loglik)
# LRT2 = 2 * (model.26.A$loglik - model.26.C$loglik)

# paste0(LRT1)
# paste0(LRT2)


# # 27 degree models 
# # Removing 22, 22.5, 26, and 28 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==28),]

# model.27.A <- asreml(fixed = sex ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # model.27.B <- asreml(fixed = sex ~ hybrid, random = ~year + sire, data=phenotypeA, family = asr_binomial(link=logit))
# # # # # # # model.27.C <- asreml(fixed = sex ~ hybrid, random = ~year + sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# vpredict(model.27.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# vpredict(model.27.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))


# summary(model.27.A)$varcomp

# LRT1 = 2 * (model.27.A$loglik - model.27.B$loglik)
# LRT2 = 2 * (model.27.A$loglik - model.27.C$loglik)

# paste0(LRT1)
# paste0(LRT2)



# # 28 degree models 
# # Removing 22, 22.5, 26, and 27 degree temperatures 

# phenotypeA <- phenotype[!(phenotype$temp==22.5 | phenotype$temp==22 | phenotype$temp==26 | phenotype$temp==27),]

# model.28.A <- asreml(fixed = sex ~ hybrid, random = ~year + sire + sire/dam, data=phenotypeA, family = asr_binomial(link=logit)) 
# # # # # # # model.28.B <- asreml(fixed = sex ~ hybrid, random = ~year + sire, data=phenotypeA, family = asr_binomial(link=logit))
# # # # # # # model.28.C <- asreml(fixed = sex ~ hybrid, random = ~year + sire/dam, data=phenotypeA, family = asr_binomial(link=logit))

# vpredict(model.28.A, heritability ~4*V2 / (V1 + V2 + V3 + V4))
# vpredict(model.28.A, maternal_effects ~(V3 - V2) / (V1 + V2 + V3 + V4))

# summary(model.28.A)$varcomp

# LRT1 = 2 * (model.28.A$loglik - model.28.B$loglik)
# LRT2 = 2 * (model.28.A$loglik - model.28.C$loglik)

# paste0(LRT1)
# paste0(LRT2)

### Generating figures

In [ ]:
theme_Publication <- function(base_size=28, base_family="Helvetica") {
      library(grid)
      library(ggthemes)
      (theme_foundation(base_size=base_size, base_family=base_family)
       + theme(plot.title = element_text(face = "bold",
                                         size = rel(1.2), hjust = 0.5),
               text = element_text(),
               panel.background = element_rect(colour = NA),
               plot.background = element_rect(colour = NA),
               panel.border = element_rect(colour = NA),
               axis.title = element_text(face = "bold",size = rel(1)),
               axis.title.y = element_text(angle=90,vjust =2),
               axis.title.x = element_text(vjust = -0.2),
               axis.text = element_text(size=rel(1)), 
               axis.line = element_line(colour="black", size=2),
               axis.ticks = element_line(),
               axis.ticks.length = unit(0.5, "cm"),
               panel.grid.major = element_blank(),
               panel.grid.minor = element_blank(),
               legend.key = element_rect(colour = NA),
               legend.text = element_text(size=30),
               legend.position = "right",
               legend.direction = "vertical",
              # makes box order legend legend.background = element_rect(colour="black"),
               legend.key.size = unit(0.2, "cm"),
               # legend.margin = unit(0, "cm"),
               legend.title = element_text(face="bold", size=40),
               plot.margin=unit(c(10,5,5,5),"mm"),
               strip.background=element_rect(colour="#f0f0f0",fill="#f0f0f0"),
               strip.text = element_text(face="bold")
          ))
      
}



scale_fill_Publication <- function(...){
      library(scales)
      discrete_scale("fill","Publication",manual_pal(values = c("#386cb0","darkorange2","#7fc97f","#ef3b2c","black","#a6cee3","#fb9a99","#984ea3","firebrick","deeppink","dodgerblue4","chartreuse4","#7e6148ff","#b09c85ff")), ...)

}

scale_colour_Publication <- function(...){
      library(scales)
      discrete_scale("colour","Publication",manual_pal(values = c("#386cb0","darkorange2","#7fc97f","#ef3b2c","black","#a6cee3","#fb9a99","#984ea3","firebrick","deeppink","dodgerblue4","chartreuse4","#7e6148ff","#b09c85ff")), ...)

}

# xlabs <- c("22.0", "22.5", "26.0", "27.0", "28.0")

### Sex ratio rxn norm plot 

# Line = ggplot(grouped_data, aes(temp, pct.male, group=sire, color=sire)) + scale_x_continuous(breaks = c(22.0, 22.5, 26.0, 27.0, 28.0), labels=xlabs) + geom_line(size=5) + geom_point(aes(), size=10, stroke=2.0) + xlab("\nIncubation temperature (°C)") + ylab("Sex ratio (proportion male)\n") + labs(colour = "Sire ID") 
# grid.arrange(Line + scale_colour_Publication() + theme_Publication())

# ggsave("reaction_norm.pdf", device = "pdf", width = 19, height = 13, units = "in", dpi=600)




# geographic rxn norm plot
# Don't need to do the grouping step 

xlabs <- c("22.0", "22.5", "24.0", "25.0", "26.0", "27.0", "28.0")
Line = ggplot(data1, aes(temp, pct.male, group=cross, color=cross)) + scale_x_continuous(breaks = c(22.0, 22.5, 24.0, 25.0, 26.0, 27.0, 28.0), labels=xlabs) + geom_line(size=5) + geom_point(aes(), size=10, stroke=2.0) + xlab("\nIncubation temperature (°C)") + ylab("Sex ratio (proportion male)\n") + labs(colour = "Cross") 
grid.arrange(Line + scale_colour_Publication() + theme_Publication())
ggsave("rxn_norm_cross.pdf", device = "pdf", width = 19, height = 13, units = "in", dpi=600)

paste0("done")



### Figure output

![alt text](hatch_cl_lsmeans.pdf "Title")

![alt text](hatch_pl_lsmeans.pdf "Title")

![alt text](hatch_mass_lsmeans.pdf "Title")

![alt text](hybrid_plot.pdf "Title")

![alt text](rxn_norm.png "Title")




In [ ]:
# Fix figures 